# 1. Weather dataset introduction

This notebook will download and format the data to be used.
The dataset selected is the [Rain in Austria](https://www.kaggle.com/jsphyg/weather-dataset-rattle-package) challenge from Kaggle. It contains about 10 years of daily weather observations from many locations across Australia. 


#### Index
1. [Download dataset from Kaggle](#1.-Download-dataset-from-Kaggle)
2. [Apply proper format to the dataset](#2.-Apply-proper-format-to-the-dataset)
3. [Dataset profiling](#3.-Dataset-profiling)

In [ ]:
!pip install pandas_profiling

In [3]:
import os
import numpy as np
import pandas as pd
from pandas_profiling import ProfileReport #!pip install pandas_profiling
import time

## 1. Download dataset from Kaggle

In [ ]:
!pip install kaggle

In [22]:
os.environ['KAGGLE_USERNAME'] = "ginesmeca" #User name in Kaggle
os.environ['KAGGLE_KEY'] = "4fab0a819b6b377c5f6fd16dede7d525" # Kaggle/ account / create key

import kaggle

kaggle.api.authenticate()
kaggle.api.dataset_download_files('jsphyg/weather-dataset-rattle-package', path='../data/01_raw', unzip=True)

## 2. Apply proper format to the dataset

### 2.1 Read the dataset

In [23]:
weather = pd.read_csv('../data/01_raw/weatherAUS.csv')
weather.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No


In [24]:
weather.dtypes

Date              object
Location          object
MinTemp          float64
MaxTemp          float64
Rainfall         float64
Evaporation      float64
Sunshine         float64
WindGustDir       object
WindGustSpeed    float64
WindDir9am        object
WindDir3pm        object
WindSpeed9am     float64
WindSpeed3pm     float64
Humidity9am      float64
Humidity3pm      float64
Pressure9am      float64
Pressure3pm      float64
Cloud9am         float64
Cloud3pm         float64
Temp9am          float64
Temp3pm          float64
RainToday         object
RainTomorrow      object
dtype: object

### 2.2 Apply the proper format

In [25]:
# Format the date
weather['Date'] = pd.to_datetime(weather.Date, format='%Y-%m-%d')

#Format RainToday by re-creating it
weather['RainToday'] = np.where(weather.Rainfall<=1.0, 0, 1)

#Remove columns we are not interested in
weather.drop('RainTomorrow',axis=1, inplace=True)

weather.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,24.0,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,0
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,22.0,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,0
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,26.0,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,0
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,9.0,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,0
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,20.0,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,0


### 2.3 Save the dataset

In [26]:
weather.to_parquet('../data/02_intermediate/weather_intermediate.parquet')

ImportError: Unable to find a usable engine; tried using: 'pyarrow', 'fastparquet'.
A suitable version of pyarrow or fastparquet is required for parquet support.
Trying to import the above resulted in these errors:
 - Missing optional dependency 'pyarrow'. pyarrow is required for parquet support. Use pip or conda to install pyarrow.
 - Missing optional dependency 'fastparquet'. fastparquet is required for parquet support. Use pip or conda to install fastparquet.

### 2.4  Creating new RainTomorrow column

### 2.4.1 Using if/for:

In [27]:
t0 = time.perf_counter()

#Creating an empty list and filling with next day RainToday values:
RainT = []
for n in range(0, len(weather) - 1):
    if weather.Location[n] == weather.Location[n+1]:
        RainT += [weather.RainToday[n+1]]
    else:
        RainT += [np.nan]

#Adding last date:
RainT += [np.nan]

#Adding RainT to weather:
weather['RainTomorrow'] = RainT

t = time.perf_counter() - t0

#Redefine to save it:
WEATHER = weather
WEATHER.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,0,0.0
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,0,0.0
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,0,0.0
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,0,0.0
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,0,0.0


In [28]:
#Running time if/for method:
t

5.338094499980798

### 2.4.2 Using np.where:

We have to reset our dataset like in 2.2

In [29]:
# Format the date
weather['Date'] = pd.to_datetime(weather.Date, format='%Y-%m-%d')

#Format RainToday by re-creating it
weather['RainToday'] = np.where(weather.Rainfall<=1.0, 0, 1)

#Remove columns we are not interested in
weather.drop('RainTomorrow',axis=1, inplace=True)

weather.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,24.0,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,0
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,22.0,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,0
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,26.0,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,0
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,9.0,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,0
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,20.0,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,0


In [44]:
t0 = time.perf_counter()

#Two datasets to compare with a day of difference
RT1 = weather[['Location', 'RainToday']]
RT2 = weather[['Location', 'RainToday']][1:len(weather)].reset_index(drop='True')
RT2.loc[len(weather) - 1] = [np.nan, np.nan]

#Creating the new column:
RainT = np.where(RT1.Location == RT2.Location, RT2.RainToday, np.nan)

#Adding RainT to weather:
weather['RainTomorrow'] = RainT

t = time.perf_counter() - t0

weather.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,0,0.0
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,0,0.0
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,0,0.0
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,0,0.0
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,0,0.0


In [45]:
#Running time np.where method:
t

0.0815565999946557

### 2.4.3 Comprobations:

Firstly, we will compare if both target columns generated are the same.

In [46]:
locs = []
for loc in weather['Location']:
    if loc not in locs:
        locs += [loc]
len(locs)

49

In [47]:
sum(weather.RainTomorrow != WEATHER.RainTomorrow)

49

The number of NaN values is equal to the number of locations, because each last date from every town is NaN. Due to we've obtained 49 errors and 49 different locations, errors are, exactly, these NaN values.

Now, we have to compare if these target columns are related with original 'RainTomorrow' column.

In [49]:
weather1 = pd.read_csv('../data/01_raw/weatherAUS.csv')
weather1['RainTomorrow'] = np.where(weather1.RainTomorrow == 'No', 0, 1)
sum(weather1.RainTomorrow != weather.RainTomorrow)

49

In summary, target columns calculated are correct. Also, due to both methods are equivalent, is far more efficient using 'np.where' method because his running time is lower.

### 3. Dataset profiling

In [84]:
profile = ProfileReport(weather, title="Weather dataset profiling report", explorative=True)
profile.to_file("../data/reporting/weather_report.html")

#profile.to_notebook_iframe()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

C:\Users\REBOLJU2\.conda\envs\py37\lib\site-packages\scipy\stats\stats.py:4594: RuntimeWarning: overflow encountered in longlong_scalars
  (2 * xtie * ytie) / m + x0 * y0 / (9 * m * (size - 2)))


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

### 3.1 Report insights